In [ ]:
import networkx as nx
import pandas as pd
from collections import defaultdict
import settings

df = pd.read_parquet("cleaned_dataframe.parquet", engine="pyarrow")
df

,word_and_gloss,language_and_translation
0,free - (social) Unconstrained.; Not imprisoned or enslaved.,Afrikaans: vrye
1,free - (social) Unconstrained.; Not imprisoned or enslaved.,Albanian: lirë (i/e)
2,free - (social) Unconstrained.; Not imprisoned or enslaved.,Amharic: ነፃ
3,free - (social) Unconstrained.; Not imprisoned or enslaved.,Arabic: حُرّ
4,free - (social) Unconstrained.; Not imprisoned or enslaved.,Arabic: حر
...,...,...
1197709,fawn response - (psychology) An overadaptation in response to a traumatic event entailing needs and wants being succumbed to those of the threat actor.,Finnish: miellyttämisreaktio
1197710,fawn response - (psychology) An overadaptation in response to a traumatic event entailing needs and wants being succumbed to those of the threat actor.,German: Bambi-Reflex
1197711,fawn response - (psychology) An overadaptation in response to a traumatic event entailing needs and wants being succumbed to those of the threat actor.,German: Unterwerfungsreaktion
1197712,"one heart, one soul - (Philippines, Catholicism) A saying that is said in every prayer, before the sign of the cross.",Latin: cor ūnum et anima ūna


In [ ]:
# Zählen der Übersetzungen pro Sprache
language_counts = df['word_and_gloss'].value_counts()

# Filtern der Sprachen, die mindestens 40 Übersetzungen haben
languages_to_keep = language_counts[language_counts >= settings.MIN_LANGUAGE_COUNT].index

# Bereinigen des DataFrames: Behalten Sie nur die Zeilen, die zu den gefilterten Sprachen gehören
df_filtered = df[df['word_and_gloss'].isin(languages_to_keep)]

df_filtered.reset_index(drop = True, inplace=True)
df_filtered

,word_and_gloss,language_and_translation
0,free - (social) Unconstrained.; Not imprisoned or enslaved.,Afrikaans: vrye
1,free - (social) Unconstrained.; Not imprisoned or enslaved.,Albanian: lirë (i/e)
2,free - (social) Unconstrained.; Not imprisoned or enslaved.,Amharic: ነፃ
3,free - (social) Unconstrained.; Not imprisoned or enslaved.,Arabic: حُرّ
4,free - (social) Unconstrained.; Not imprisoned or enslaved.,Arabic: حر
...,...,...
559343,call the fire department - (US) Call the emergency service that specializes in extinguishing fires.,Tagalog: tawag ka ng bumbero
559344,call the fire department - (US) Call the emergency service that specializes in extinguishing fires.,Turkish: itfaiyeyi çağırin
559345,call the fire department - (US) Call the emergency service that specializes in extinguishing fires.,Turkish: itfaiyeyi çağır
559346,call the fire department - (US) Call the emergency service that specializes in extinguishing fires.,Ukrainian: ви́кличте поже́жників


In [29]:
len(df_filtered['word_and_gloss'].unique())

6911

In [30]:
#filtered_df = df[df['word_and_gloss'].str.startswith('head -')]
#filtered_df.reset_index(drop=True, inplace=True)
#filtered_df

In [31]:
# Anzahl der eindeutigen Konzepte und Übersetzungen
#num_concepts = filtered_df['word_and_gloss'].nunique()
#num_translations = filtered_df['language_and_translation'].nunique()

#print(f"Anzahl der eindeutigen Konzepte: {num_concepts}")
#print(f"Anzahl der eindeutigen Übersetzungen: {num_translations}")

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

# Erstellen einer Co-Occurrence-Matrix
#co_occurrence_matrix = pd.crosstab(filtered_df['word_and_gloss'], filtered_df['language_and_translation'])
#co_occurrence_matrix.head()

In [33]:
from sklearn.metrics.pairwise import cosine_similarity
#concepts = filtered_df['word_and_gloss'].unique()

# Berechnen der Kosinus-Ähnlichkeit
#similarity_matrix = cosine_similarity(co_occurrence_matrix)
#similarity_df = pd.DataFrame(similarity_matrix, index=concepts, columns=concepts)
#similarity_df.head()

In [34]:
#distance_matrix = 1 - similarity_matrix
#distance_df = pd.DataFrame(distance_matrix, index=concepts, columns=concepts)
#distance_df.head()

In [35]:
import seaborn as sns
import matplotlib.pyplot as plt

# Heatmap der Distanzmatrix
#plt.figure(figsize=(8, 6))
#sns.heatmap(distance_df, annot=True, cmap='viridis', fmt='.2f')
#plt.title('Distanzmatrix zwischen Konzepten')
#plt.show()

## New Try

In [ ]:
translation_counts = df_filtered["language_and_translation"].value_counts()


# Filtern der Uebersetzungen die mehr als einmal vorkommen
translations_to_keep = translation_counts[translation_counts > 1].index

df_filtered = df_filtered[df_filtered['language_and_translation'].isin(translations_to_keep)]

df_filtered.reset_index(drop = True, inplace=True)
pd.set_option("display.max_rows", 20)
pd.set_option('display.max_colwidth', None)
df_filtered = df_filtered.drop_duplicates()
df_filtered

# TODO add pos to language_and_translation because Arabic: حر can be noun or verb and should therefore be seperated


,word_and_gloss,language_and_translation
0,free - (social) Unconstrained.; Not imprisoned or enslaved.,Arabic: حُرّ
1,free - (social) Unconstrained.; Not imprisoned or enslaved.,Arabic: حر
3,free - (social) Unconstrained.; Not imprisoned or enslaved.,Armenian: ազատ
4,free - (social) Unconstrained.; Not imprisoned or enslaved.,Assamese: মুকলি
5,free - (social) Unconstrained.; Not imprisoned or enslaved.,Asturian: llibre
6,free - (social) Unconstrained.; Not imprisoned or enslaved.,Azerbaijani: azad
7,free - (social) Unconstrained.; Not imprisoned or enslaved.,Bambara: hɔrɔn
8,free - (social) Unconstrained.; Not imprisoned or enslaved.,Bashkir: ирекле
9,free - (social) Unconstrained.; Not imprisoned or enslaved.,Catalan: lliure
10,free - (social) Unconstrained.; Not imprisoned or enslaved.,Chinese Cantonese: 自由
